# 词嵌入（word2vec）
:label:`sec_word2vec`


自然语言是一个用来表达意义的复杂系统。
在这个系统中，词是意义的基本单位。
顾名思义，
*词向量*是用来表示词的向量，
也可以被视为词的特征向量或表示形式。
将词映射到实向量的技术
称为*词嵌入*。
近年来，
词嵌入逐渐成为
自然语言处理的基础知识。


## 独热向量不是一个好的选择

我们在:numref:`sec_rnn-scratch`中使用独热向量来表示词（字符即为词）。
假设词典中不同词的数量（词典大小）为$N$，
每个词对应
从$0$到$N-1$的不同整数（索引）。
为了获得索引为$i$的任何词的独热向量表示，
我们创建一个长度为$N$、所有元素为0的向量，
并将位置$i$处的元素设置为1。
这样，每个词都被表示为长度为$N$的向量，并且
可以直接用于神经网络。


尽管独热词向量易于构造，
但它们通常不是一个好的选择。
主要原因是独热词向量不能准确地表达不同词之间的相似性，例如我们常用的*余弦相似度*。
对于向量$\mathbf{x}, \mathbf{y} \in \mathbb{R}^d$，它们的余弦相似度是它们之间夹角的余弦值：


$$\frac{\mathbf{x}^\top \mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|} \in [-1, 1].$$


由于任意两个不同词的独热向量之间的余弦相似度为0，
独热向量无法编码词之间的相似性。


## 自监督word2vec

[Word2vec](https://code.google.com/archive/p/word2vec/)工具被提出以解决上述问题。
它将每个词映射到固定长度的向量，这些向量可以更好地表达不同词之间的相似性和类比关系。
Word2vec工具包含两个模型，即*skip-gram* :cite:`Mikolov.Sutskever.Chen.ea.2013` 和 *连续词袋* (CBOW) :cite:`Mikolov.Chen.Corrado.ea.2013`。
为了获得语义上有意义的表示，
它们的训练依赖于
可以从数据中无标签部分获取的
条件概率
这可以看作是在语料库中
使用一些词周围的词来预测某些词。
由于监督来自没有标签的数据，
skip-gram和连续词袋都是自监督模型。

接下来，我们将介绍这两个模型及其训练方法。


## Skip-Gram 模型
:label:`subsec_skip-gram`

*Skip-gram* 模型假设一个词可以用来生成其在文本序列中的周围词。
以文本序列"the", "man", "loves", "his", "son"为例。
让我们选择"loves"作为*中心词*并将上下文窗口大小设为2。
如:numref:`fig_skip_gram`所示，
给定中心词"loves"，
skip-gram模型考虑
生成*上下文词*"the", "man", "his", 和 "son"的条件概率，
这些词距离中心词不超过2个词：

$$P(\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}\mid\textrm{"loves"}).$$

假设
给定中心词时，上下文词是独立生成的（即条件独立）。
在这种情况下，上述条件概率
可以重写为

$$P(\textrm{"the"}\mid\textrm{"loves"})\cdot P(\textrm{"man"}\mid\textrm{"loves"})\cdot P(\textrm{"his"}\mid\textrm{"loves"})\cdot P(\textrm{"son"}\mid\textrm{"loves"}).$$

![Skip-gram 模型考虑了给定中心词生成周围上下文词的条件概率。](../img/skip-gram.svg)
:label:`fig_skip_gram`

在skip-gram模型中，每个词
都有两个$d$维向量表示
用于计算条件概率。
更具体地说，
对于词典中索引为$i$的任何词，
用$\mathbf{v}_i\in\mathbb{R}^d$
和$\mathbf{u}_i\in\mathbb{R}^d$
分别表示
当其用作*中心词*和*上下文词*时的两个向量。
给定中心词$w_c$（词典中索引为$c$），生成任何上下文词$w_o$（词典中索引为$o$）的条件概率可以通过向量点积上的softmax操作建模：


$$P(w_o \mid w_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)},$$
:eqlabel:`eq_skip-gram-softmax`

其中词汇索引集$\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$。
给定长度为$T$的文本序列，其中时间步$t$处的词表示为$w^{(t)}$。
假设
给定任何中心词时，上下文词是独立生成的。
对于上下文窗口大小$m$，
skip-gram模型的似然函数
是生成所有上下文词
给定任何中心词的概率：


$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),$$

其中任何小于$1$或大于$T$的时间步都可以省略。

### 训练

Skip-gram模型的参数是词汇表中每个词的中心词向量和上下文词向量。
在训练中，我们通过最大化似然函数（即最大似然估计）来学习模型参数。这等价于最小化以下损失函数：

$$ - \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \textrm{log}\, P(w^{(t+j)} \mid w^{(t)}).$$

当使用随机梯度下降法最小化损失时，
在每次迭代中
我们可以
随机采样一个较短的子序列来计算该子序列的（随机）梯度以更新模型参数。
为了计算这个（随机）梯度，
我们需要获得
关于中心词向量和上下文词向量的
对数条件概率的梯度。
一般而言，根据:eqref:`eq_skip-gram-softmax`
涉及任何中心词$w_c$和
上下文词$w_o$的对数条件概率为


$$\log P(w_o \mid w_c) =\mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right).$$
:eqlabel:`eq_skip-gram-log`

通过对数求导，我们可以获得其相对于中心词向量$\mathbf{v}_c$的梯度为

$$\begin{aligned}\frac{\partial \textrm{log}\, P(w_o \mid w_c)}{\partial \mathbf{v}_c}&= \mathbf{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c)\mathbf{u}_j}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}\\&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\exp(\mathbf{u}_j^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}\right) \mathbf{u}_j\\&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.\end{aligned}$$
:eqlabel:`eq_skip-gram-grad`


请注意，在:eqref:`eq_skip-gram-grad`中的计算需要词典中所有词与$w_c$作为中心词的条件概率。
其他词向量的梯度可以以相同的方式获得。


训练后，对于词典中索引为$i$的任何词，我们同时获得了词向量
$\mathbf{v}_i$（作为中心词）和$\mathbf{u}_i$（作为上下文词）。
在自然语言处理应用中，通常使用skip-gram模型的中心词向量
作为词的表示。


## 连续词袋（CBOW）模型


*连续词袋*（CBOW）模型类似于skip-gram模型。
与skip-gram模型的主要区别在于
连续词袋模型
假设中心词是基于其在文本序列中的周围上下文词生成的。
例如，
在同一文本序列"the", "man", "loves", "his", 和 "son"中，以"loves"为中心词且上下文窗口大小为2，
连续词袋模型
考虑
基于上下文词"the", "man", "his" 和 "son"生成中心词"loves"的条件概率（如:numref:`fig_cbow`所示），即

$$P(\textrm{"loves"}\mid\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}).$$

![连续词袋模型考虑了给定其周围上下文词生成中心词的条件概率。](../img/cbow.svg)
:label:`fig_cbow`


由于连续词袋模型中有多个上下文词，
在计算条件概率时
这些上下文词向量会被平均。
具体来说，
对于词典中索引为$i$的任何词，
用$\mathbf{v}_i\in\mathbb{R}^d$
和$\mathbf{u}_i\in\mathbb{R}^d$
分别表示
当其用作*上下文词*和*中心词*时的两个向量
（含义在skip-gram模型中相反）。
生成任何中心词$w_c$（词典中索引为$c$）的条件概率
给定其周围的上下文词$w_{o_1}, \ldots, w_{o_{2m}}$（词典中索引为$o_1, \ldots, o_{2m}$）可以建模为



$$P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\exp\left(\frac{1}{2m}\mathbf{u}_c^\top (\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \exp\left(\frac{1}{2m}\mathbf{u}_i^\top (\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}}) \right)}.$$
:eqlabel:`fig_cbow-full`


为了简洁起见，令$\mathcal{W}_o= \{w_{o_1}, \ldots, w_{o_{2m}}\}$ 和 $\bar{\mathbf{v}}_o = \left(\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}} \right)/(2m)$。则:eqref:`fig_cbow-full`可以简化为

$$P(w_c \mid \mathcal{W}_o) = \frac{\exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)}.$$

给定长度为$T$的文本序列，其中时间步$t$处的词表示为$w^{(t)}$。
对于上下文窗口大小$m$，
连续词袋模型的似然函数
是生成所有中心词
给定其上下文词的概率：


$$ \prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).$$

### 训练

训练连续词袋模型
几乎与
训练skip-gram模型相同。
连续词袋模型的最大似然估计等价于最小化以下损失函数：



$$  -\sum_{t=1}^T  \textrm{log}\, P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).$$

注意到

$$\log\,P(w_c \mid \mathcal{W}_o) = \mathbf{u}_c^\top \bar{\mathbf{v}}_o - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)\right).$$

通过对数求导，我们可以获得其相对于任何上下文词向量$\mathbf{v}_{o_i}$（$i = 1, \ldots, 2m$）的梯度为


$$\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} = \frac{1}{2m} \left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} \frac{\exp(\mathbf{u}_j^\top \bar{\mathbf{v}}_o)\mathbf{u}_j}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \bar{\mathbf{v}}_o)} \right) = \frac{1}{2m}\left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right).$$
:eqlabel:`eq_cbow-gradient`


其他词向量的梯度可以以相同方式获得。
与skip-gram模型不同，
连续词袋模型
通常
使用上下文词向量作为词的表示。




## 总结

* 词向量是用于表示词的向量，也可以视为词的特征向量或表示形式。将词映射到实向量的技术称为词嵌入。
* Word2vec工具既包含skip-gram模型也包含连续词袋模型。
* Skip-gram模型假设一个词可以用来生成其在文本序列中的周围词；而连续词袋模型假设一个中心词是基于其周围的上下文词生成的。



## 练习

1. 计算每个梯度的计算复杂度是多少？如果词典非常大，可能会出现什么问题？
1. 英语中的一些固定短语由多个词组成，例如"new york"。如何训练它们的词向量？提示：参见论文:cite:`Mikolov.Sutskever.Chen.ea.2013`的第4节。
1. 以skip-gram模型为例，反思word2vec的设计。在skip-gram模型中，两个词向量的点积与余弦相似度之间有什么关系？为什么一对语义相似的词的词向量（通过skip-gram模型训练）的余弦相似度可能很高？

[讨论](https://discuss.d2l.ai/t/381)